In [ ]:
import json
import torch
from smile import Smile
import matplotlib.pyplot as plt
import numpy as np
import glob
import os
from tqdm import tqdm
plt.style.use("seaborn")

In [ ]:
name = "spx_vix_small_sept"
path = f"/home/ziyed/clusters/della-gpu/neural-spx-vix-calibration/src/checkpoints/{name}"

files = glob.glob(f"{path}/*.json")
files = sorted(files, key=os.path.getmtime)
last = files[-1]

In [ ]:
last

In [ ]:
def get_smile(maturity, smile):
    data = {
        "strikes" : torch.tensor(smile["strikes"]),
        "fwd" : torch.tensor(smile["fwd"]),
        "fwd_ask" : torch.tensor(smile["fwd_ask"]),
        "fwd_bid" : torch.tensor(smile["fwd_bid"]),
        "mids" : torch.tensor(smile["mids"]),
        "asks" : torch.tensor(smile["asks"]),
        "bids" : torch.tensor(smile["bids"]),
        "calls" : torch.tensor(smile["calls"]),
        "puts" : torch.tensor(smile["puts"])

    }
    return Smile(smile["name"], maturity, data)

In [ ]:
def plot(file):
    with open(file) as f:
        checkpoint = json.load(f)
    maturities = checkpoint["smiles"]
    plt.figure(figsize=(10, 5 * len(maturities)))

    for k, (maturity, data) in enumerate(checkpoint["smiles"].items()):
        for i, (instrument, smile) in enumerate(data.items()):
            plt.subplot(len(maturities), 1, k + 1)
            model, market = smile["model"], smile["market"]
            model = get_smile(maturity, model)
            market = get_smile(maturity, market)
            model.plot(color="black")
            market.plot(color="red")
            plt.title(f"{instrument.capitalize()} {maturity}")
    plt.show()

In [ ]:
plot(files[-1])